<a href="https://colab.research.google.com/github/fuzailpalnak/pestoAssignment/blob/main/pestoAssignmentInference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets
! pip install accelerate==0.31.0

In [3]:
from transformers import EncoderDecoderModel, BertTokenizer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import requests
import zipfile
import io
import os

# Define the URL of the ZIP file
url = 'https://github.com/fuzailpalnak/pestoAssignment/releases/download/v0.0.1/saved_model.zip'

# Download the ZIP file into memory
response = requests.get(url)
if response.status_code == 200:
    # Create a BytesIO object from the response content
    zip_file = io.BytesIO(response.content)

    # Extract the ZIP file into a temporary directory
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        temp_dir = '/content'
        zip_ref.extractall(temp_dir)

In [8]:
test_query = "I accidently order the wrong product"

In [ ]:
# Load model and tokenizer for prediction
loaded_tokenizer = BertTokenizer.from_pretrained("/content/saved_model")
loaded_model = EncoderDecoderModel.from_pretrained("/content/saved_model")

# Ensure the loaded model has the correct configuration
loaded_model.generation_config.pad_token_id = loaded_tokenizer.pad_token_id
loaded_model.generation_config.eos_token_id = loaded_tokenizer.sep_token_id
loaded_model.generation_config.decoder_start_token_id = loaded_tokenizer.cls_token_id

t = BertTokenizer.from_pretrained('bert-base-uncased')
# Function to generate response
def generate_response(query):
    inputs = loaded_tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=128)
    output_ids = loaded_model.generate(inputs.input_ids, max_length=128)
    return loaded_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Test the loaded model
test_query = "I accidently order the wrong product"
print(f"Query: {test_query}")
print(f"Response: {generate_response(test_query)}")

In [ ]:
while True:
    user_input = input("Enter Query")
    if user_input.lower()=="exit":
      break
    print(generate_response(user_input))
